In [1]:
%reload_ext autoreload
%autoreload 2

import os
import sys
sys.path.append(os.path.join(os.environ['REPO_DIR'], 'utilities'))
from visualization_utilities import *
from metadata import *
from utilities2015 import *

Setting environment for Local Macbook Pro


In [2]:
from pandas import *

# Check structure annotations

In [4]:
structures_df = DataManager.load_annotation_v4(stack='CHATM3', by_human=True, 
timestamp='latest', suffix='structures')

aws s3 cp --recursive "s3://mousebrainatlas-data/CSHL_labelings_v3/CHATM3" "/home/yuncong/CSHL_labelings_v3/CHATM3" --exclude "*" --include "*structures*"
0
latest timestamp:  05102018102201


In [21]:
for sid, structure in structures_df.iterrows():
    print structure['name'], structure['side'], bp.unpack_ndarray_str(structure['volume']).shape

Pn L (129, 143, 145)
SNR L (252, 157, 199)
3N R (45, 81, 37)
Sp5O L (156, 338, 69)
10N L (54, 189, 84)
LC R (98, 109, 39)
SNC R (175, 98, 110)
VLL R (292, 176, 119)
PBG R (117, 66, 74)
SNC L (170, 96, 122)
Tz R (63, 133, 77)
4N R (35, 29, 27)
PBG L (127, 66, 43)
VLL L (285, 192, 119)
AP S (46, 91, 54)
Sp5C L (242, 301, 152)
RMC L (95, 109, 109)
LC L (103, 105, 45)
DC R (106, 176, 184)
Amb L (36, 87, 18)
LRt R (89, 199, 105)
3N L (62, 79, 37)
6N R (38, 36, 33)
Sp5I L (156, 269, 80)
10N R (81, 183, 67)
SNR R (251, 154, 199)
DC L (106, 176, 184)
VCA L (138, 162, 94)
VCP L (164, 109, 94)
IC S (311, 208, 438)
5N R (119, 100, 74)
6N L (38, 36, 33)
Tz L (55, 138, 77)
7N R (79, 119, 152)
VCP R (173, 109, 74)
12N S (86, 168, 108)
7N L (97, 129, 118)
7n L (155, 106, 188)
4N L (35, 29, 27)
RMC R (95, 109, 109)
SC S (305, 319, 462)
VCA R (149, 162, 85)
Sp5I R (166, 270, 80)
5N L (119, 100, 74)
RtTg S (165, 178, 233)
Pn R (129, 143, 145)
Amb R (37, 89, 18)
LRt L (89, 199, 105)
Sp5C R (210, 325, 152

In [22]:
def get_edit_transform(structures_df):
    for struct_id, structure_info in structures_df.iterrows():
        if structure_info['volume'] is not None:
            vol = bp.unpack_ndarray_str(structure_info['volume'])
            name_s = compose_label(structure_info['name'], side=structure_info['side'])
            print name_s, [ed['type'] for ed in structure_info['edits']]

            global_tf_from_wholebrain_to_wholebrain_volResol = np.eye(4)
            local_tf_from_wholebrain_to_wholebrain_volResol = np.eye(4)
            composed_tf_from_wholebrain_to_wholebrain_volResol = np.eye(4)
            for edit in structure_info['edits']:
                edit_tf_from_wholebrain_to_wholebrain_volRes = consolidate(edit['transform'])
                if edit['type'].startswith('global'): #  global_rotate3d or global_shift3d
                    global_tf_from_wholebrain_to_wholebrain_volResol = \
                    np.dot(edit_tf_from_wholebrain_to_wholebrain_volRes, global_tf_from_wholebrain_to_wholebrain_volResol)
                elif edit['type'].startswith('prob'):
                    local_tf_from_wholebrain_to_wholebrain_volResol = \
                    np.dot(edit_tf_from_wholebrain_to_wholebrain_volRes, local_tf_from_wholebrain_to_wholebrain_volResol)                    
                    
                composed_tf_from_wholebrain_to_wholebrain_volResol = \
                np.dot(edit_tf_from_wholebrain_to_wholebrain_volRes, composed_tf_from_wholebrain_to_wholebrain_volResol)

            print global_tf_from_wholebrain_to_wholebrain_volResol.reshape((4,4))[:3]
            print local_tf_from_wholebrain_to_wholebrain_volResol.reshape((4,4))[:3]
            print composed_tf_from_wholebrain_to_wholebrain_volResol.reshape((4,4))[:3]

In [5]:
get_edit_transform(structures_df)

SNR_R ['idle', 'global_shift3d', 'global_shift3d', 'global_shift3d']
[[  1.           0.           0.          42.810525  ]
 [  0.           1.           0.         -10.15347555]
 [  0.           0.           1.          18.29485642]]
[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]]
[[1.00000000e+00 0.00000000e+00 0.00000000e+00 1.17958358e+03]
 [0.00000000e+00 1.00000000e+00 0.00000000e+00 5.97622082e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00 4.30182702e+02]]
SNR_L ['idle', 'global_shift3d', 'global_shift3d', 'global_shift3d']
[[  1.           0.           0.          42.810525  ]
 [  0.           1.           0.         -10.15347555]
 [  0.           0.           1.          18.29485642]]
[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]]
[[1.00000000e+00 0.00000000e+00 0.00000000e+00 1.17958358e+03]
 [0.00000000e+00 1.00000000e+00 0.00000000e+00 5.97622082e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00 4.30182702e+02]]


# 2-D Contours

In [6]:
stack = 'MD589'

In [7]:
labeling_dir = '/home/yuncong/CSHL_labelings_v3/' + stack

In [8]:
data = load_hdf_v2(os.path.join(labeling_dir, 'MD589_annotation_contours_04192018111821.hdf'))

In [5]:
annotations = {}

for fp in sorted(os.listdir(labeling_dir)):
    if 'contour' in fp:
        data = load_hdf_v2(os.path.join(labeling_dir, fp))
        print fp
        for ind, row in data.iterrows():
            print row['name'], row['section']
            annotations[(row['name'], row['section'])] = row    

MD585_annotation_contours_04192018113554.hdf
SC 249
IC 180
DC 130
10N 216
IO 230
SC 195
IC 232
5N 141
SolIM 175
7n 156
SC 172
VCP 337
Bar 180
IC 174
IO 201
Pn 193
7n 296
RMC 255
Sp5C 151
SC 251
Tz 235
Pn 205
Sp5I 105
SNC 295
SNR 314
8n 103
VCA 100
7n 232
7n 188
8n 101
SC 218
PBG 138
IC 212
RtTg 205
IC 291
7n 167
8n 102
PBG 125
5N 153
DC 321
Tz 246
Sp5I 111
5N 151
RtTg 212
RtTg 235
SNC 145
7n 154
VCA 87
RtTg 207
VLL 143
Pn 185
LPB 142
6N 228
LC 259
VCP 105
SNC 141
SC 308
7N 272
RMC 191
SC 292
RtTg 189
Sp5C 154
Sol 170
RtTg 195
SC 317
Sp5I 301
RtTg 245
SC 273
VCP 104
7n 270
IC 184
Pn 236
Pn 202
Sp5C 114
Tz 255
12N 202
7n 288
LC 258
SNC 273
7n 170
SC 164
Sp5I 296
12N 196
7N 258
VCP 102
DC 88
7N 177
5N 290
SC 290
LRt 284
Sp5C 117
6N 189
IC 166
Sp5C 113
IO 241
DC 291
SC 227
IC 239
VCA 83
SNR 306
SpVe 149
SNC 275
5N 286
Sp5I 309
Pn 201
Sp5C 153
SNR 304
SNC 268
Pn 195
VCP 92
SNR 300
Tz 206
SNC 308
Sp5C 147
7n 245
Tz 198
Tz 186
LVe 141
8n 314
SNC 277
7N 268
RtTg 215
VLL 131
AP 196
VCP 322
VLL 

DC 90
IC 210
SC 148
Amb 269
SNR 157
Amb 149
VCA 110
RMC 207
SNR 277
VCA 335
Pn 206
Sp5I 104
DC 305
RtTg 184
Sp5C 123
IC 287
10N 204
VCA 104
Sp5O 310
10N 194
IC 271
5N 147
SuVe 138
SNC 298
SC 207
6N 186
Sp5C 141
7N 171
7n 257
IC 183
RtTg 180
IC 178
SNC 263
Pr5DM 119
SC 166
AP 191
SNC 304
7n 139
Sp5O 280
Sp5I 126
IC 234
Sp5I 124
7N 266
LPB 143
SNC 168
SC 158
7N 180
Tz 230
7n 251
IC 249
IC 167
RtTg 203
SC 222
IC 303
VCA 324
IC 298
SNC 162
Sp5C 129
SuVe 141
SNR 318
RMC 239
7N 264
IC 247
SC 190
VLL 137
8n 305
SNR 305
Pn 241
Pn 186
VCA 95
DC 119
3N 234
IC 200
SC 139
12N 192
10N 191
Pr5DM 309
SNR 175
12N 198
4N 205
VCA 342
Tz 253
VCA 101
IC 289
VCA 340
VCP 96
IC 268
LRt 141
SC 316
IO 200
SNR 271
SC 233
Sp5O 117
SC 240
SNR 275
DC 110
SC 246
SC 163
VCP 315
SC 268
VCP 94
PBG 129
7N 263
DC 124
12N 203
LC 260
SC 257
VLL 135
SNC 283
DC 304
7N 174
7n 150
8n 315
VCP 317
SC 286
Sp5C 139
Sp5O 301
SC 266
Tz 238
IO 199
VLL 304
Tz 188
SC 313
RMC 274
Tz 233
Sp5I 306
5N 155
PBG 133
Pr5DM 306
3N 228
VCP 334


Sp5I 129
SNC 273
IC 281
Sp5C 166
Sp5C 282
Sp5I 122
IC 162
SC 143
VCA 326
Pn 204
RtTg 219
Sp5C 155
4N 232
RMC 239
RtTg 259
12N 189
SC 192
Sp5C 109
AP 202
DC 91
Pn 276
SC 286
LRt 284
10N 206
SNC 286
RMC 203
VCP 331
Tz 237
Sp5O 117
SC 262
SNR 147
SNC 271
12N 212
Sp5I 296
IC 163
SNR 184
IC 180
SC 283
SNR 268
DC 330
Sol 164
Sp5I 288
7N 169
7n 171
8n 107
VCA 323
Sp5C 130
IC 251
8n 109
DC 293
DC 313
SNR 320
Sp5C 153
IC 237
SC 185
RMC 192
RtTg 250
Pn 261
8n 307
SNR 133
5N 141
10N 185
Sp5O 106
6N 225
SNC 307
DC 134
IC 152
Sp5I 103
SNR 311
7n 282
7n 239
RtTg 242
PBG 126
SNR 178
AP 191
DC 311
SC 277
5N 145
Pn 215
5N 275
SC 158
Pn 268
12N 211
7N 162
IC 137
VLL 301
7n 249
Pn 241
VLL 145
7N 170
Pn 242
5N 152
12N 188
SC 242
RtTg 177
SNC 290
5N 151
DC 305
IC 211
SNR 307
7N 175
IO 196
10N 180
VLL 303
IC 146
SC 238
3N 226
SC 164
VLL 310
RtTg 253
Sp5I 308
DC 104
SNR 282
AP 197
7n 176
SC 296
Sp5O 310
10N 188
Pn 209
Sp5O 113
RMC 256
Pn 184
RtTg 245
SNR 149
Cb 270
RMC 208
SNR 163
IC 256
7N 283
5N 282
IC 149

SC 279
7n 135
PBG 137
IO 195
Pr5DM 114
Sp5O 301
SC 178
SC 303
LC 261
7n 165
Sp5O 121
RtTg 249
Sp5C 110
7N 162
Sp5I 129
SC 235
DC 290
10N 190
IC 159
SC 143
IC 174
SC 207
SC 270
RtTg 214
7n 174
SNC 269
DC 114
Tz 205
5N 286
SC 246
SC 300
SNR 170
DC 306
SC 141
SC 198
SC 167
SC 235
RtTg 194
SC 316
SC 206
Pr5DM 117
7n 294
SNC 283
Pn 185
8n 103
VLL 149
SC 314
8n 307
VCP 343
SC 214
7n 272
Tz 237
SNC 270
VCA 107
SNR 169
IC 230
SC 218
SC 232
LRt 147
Sp5C 139
LC 259
IO 202
SC 275
VCA 337
SC 147
DC 117
SC 234
7n 139
SC 179
MVePC 170
VCP 318
7N 167
SC 212
8n 102
RMC 247
5N 140
IC 300
SC 252
7N 155
SNR 293
SC 168
IC 183
IC 202
SC 200
RtTg 216
SNC 148
SC 230
VCA 91
SC 315
Pn 194
VCP 97
7N 261
SC 281
Sp5O 128
PBG 129
SC 153
SC 141
LPB 144
7n 167
RtTg 242
SC 189
LC 169
SC 206
IC 197
SC 307
PBG 314
12N 194
DC 324
7N 255
SC 270
SC 149
Sp5C 134
SC 249
SC 160
SC 240
RtTg 227
SC 254
IC 185
12N 187
Pn 230
IC 219
SC 277
8n 113
IC 146
IO 229
7n 296
7N 252
SC 238
IC 286
Amb 275
DC 135
IC 282
SC 220
SC 266
7n 24

SC 155
SC 150
SC 250
SNC 290
SC 283
MVeMC 170
VLL 155
IO 231
10N 219
10N 208
12N 193
SC 254
SC 254
SC 280
10N 215
IC 243
VCP 94
Cu 170
Pn 259
SC 257
SC 285
SC 256
Pn 170
5N 141
SC 309
SpVe 170
SC 178
SNR 180
DC 301
SC 214
SNR 306
Pr5DM 311
6N 223
SNR 137
SC 314
IC 156
RMC 202
SNR 143
4N 211
SC 150
RMC 243
SC 229
8n 116
SC 167
VLL 301
SNR 173
Pn 277
SC 219
SC 279
SC 270
SC 313
Tz 202
SC 187
VCP 321
SC 144
VCA 327
IC 167
SC 226
Sp5I 305
SNR 183
SNC 279
VCA 109
SC 304
Tz 195
SC 186
Sp5C 133
SC 237
IC 268
10N 222
RMC 237
4N 213
Sp5O 117
SC 171
SC 162
3N 210
VCP 324
IC 294
SC 280
SC 255
Sp5C 296
SC 261
4N 231
IC 291
7n 250
SNC 144
SC 221
PBG 323
Sp5C 151
SC 177
RMC 258
LVe 141
SC 227
Pn 270
7n 263
DC 320
10N 188
SC 202
SNC 303
SC 158
Sp5O 127
IC 295
Sp5C 125
SC 161
8n 121
SC 191
SC 137
7N 148
10N 181
VCP 344
7n 151
SC 148
SNC 278
SC 173
SC 206
10N 214
5N 147
SC 207
SC 213
VLL 153
Sol 160
SC 148
10N 191
SC 243
SC 211
Pn 261
7n 283
Sp5I 121
DC 333
AP 193
Tz 194
7n 141
SC 279
7n 288
SC 287
VLL

...

Tz 193
Pr5DM 305
Tz 184
7n 233
IC 298
RMC 201
SNR 314
SNR 306
LPB 144
5N 143
VCP 101
RtTg 214
IC 138
IO 240
SC 219
MVePC 167
DC 130
IO 236
VLL 127
7n 232
DC 310
SNR 292
DC 304
LPB 134
7N 177
SC 213
12N 218
SC 313
SNR 162
10N 195
VLL 290
Sp5C 127
RtTg 193
Sp5O 301
RtTg 238
SNC 159
Tz 195
IC 264
5N 291
SNC 272
3N 232
DC 100
3N 233
IC 256
Sp5I 102
3N 234
RtTg 176
IC 280
SNR 323
12N 207
Sol 160
IC 244
4N 231
SNR 264
SNC 296
6N 230
Sp5O 292
Amb 273
SC 244
8n 106
7N 286
Pn 253
SC 300
Sp5I 296
SNC 154
LC 175
Sp5C 292
DC 119
7n 256
Sp5O 309
8n 306
RMC 188
Sp5I 108
Tz 238
IC 189
IC 131
VCA 105
Pn 165
DC 315
Tz 237
SNR 304
RMC 191
VCA 85
IC 196
IC 172
3N 221
Pn 205
IC 212
4N 235
7N 161
SC 264
IC 239
IC 166
7N 279
8n 119
SC 185
7n 167
RtTg 179
Sp5C 146
RtTg 226
SC 283
5N 288
Cb 180
DC 330
4N 213
RtTg 256
8n 307
12N 200
SNR 146
Pn 269
Pn 172
VLL 148
Tz 197
RtTg 189
DC 331
7n 179
Cb 300
7N 157
RMC 237
RtTg 259
SNC 303
SNC 260
VCA 87
RtTg 258
VLL 128
SC 286
IC 207
10N 181
RtTg 257
IC 143
Sp5I 120
Sp

IO 236
3N 213
LPB 136
RMC 243
IO 237
VLL 288
LRt 283
RtTg 227
Amb 149
7n 275
LRt 170
DC 336
SC 269
IC 265
VCA 84
Sp5C 145
RtTg 195
Pn 253
7N 150
SNC 158
SNC 302
Pn 196
IC 254
VCA 350
Pn 250
7n 254
IC 183
VLL 292
Pn 255
5N 152
Pn 187
SNR 128
LRt 278
Sp5C 141
IC 264
Sp5C 154
10N 212
7n 187
Sp5I 110
IO 226
SNR 285
7n 245
IC 179
7n 164
DC 98
RMC 265
10N 218
SNC 147
DC 311
DC 306
Pn 171
LC 258
Sp5C 292
SC 138
Pn 277
SNC 282
IC 260
RtTg 239
IC 165
SNR 273
SNC 151
SNR 127
IC 277
5N 147
VCP 333
PBG 127
3N 211
SNC 261
10N 217
DC 324
SNC 269
Pn 189
7N 153
VCP 314
RMC 201
SNR 166
IC 175
RtTg 237
SC 160
Pn 260
SC 156
AP 193
RtTg 197
Sp5C 296
8n 310
SC 150
7N 173
SC 229
7N 160
LVe 136
12N 198
VCA 334
7N 283
IC 234
6N 195
SC 280
SNR 318
7n 284
Sp5O 119
SNC 284
6N 192
Cu 160
Tz 203
Pr5DM 310
RMC 193
Sp5I 105
SNR 269
VCA 103
VCA 86
IC 307
SNR 266
IC 172
VLL 289
Sp5C 114
SC 219
7N 290
7N 169
SNC 141
7N 144
RMC 242
SNR 286
Tz 192
IC 174
SNR 292
Pn 234
Pr5DM 115
SNC 155
Sp5I 306
VCA 325
SC 153
3N 232
VCA

8n 107
12N 205
RtTg 235
SC 250
SNR 169
Tz 242
RtTg 216
7n 249
12N 195
RMC 249
IC 167
SC 220
7n 227
Sp5I 307
VCP 96
RMC 265
10N 212
Sp5O 111
DC 325
DC 285
VCP 339
VLL 289
8n 306
RtTg 232
SC 162
4N 207
SNC 266
3N 225
IO 234
10N 203
7N 281
4N 212
3N 226
VLL 131
Sp5I 310
Sp5C 112
SNR 160
LC 261
SC 284
Sp5I 127
12N 216
7N 276
SC 294
SNR 321
7N 261
SC 282
Pr5DM 314
VLL 304
DC 296
6N 229
10N 220
SC 177
SNR 180
Pn 177
Pn 176
SC 167
VCA 330
SuVe 145
IO 201
IC 295
Sp5O 314
PBG 319
DC 89
3N 214
AP 193
SC 265
IC 267
SC 158
IC 166
RtTg 198
Sp5C 123
7n 259
DC 112
IO 228
12N 206
DC 291
DC 307
Sp5C 296
IC 152
Pr5DM 307
IC 192
SC 274
Tz 246
VLL 143
VCP 313
VCP 316
MVePC 170
SC 185
DC 319
SNC 281
SC 176
PBG 324
SC 281
IC 258
Sp5I 128
RMC 272
Sp5I 296
DC 108
Sp5C 147
LPB 137
AP 204
Pr5DM 120
Pn 236
SC 267
SNR 271
VLL 152
Pn 201
IC 255
SC 252
IC 236
SC 300
DC 292
IC 217
DC 297
SC 149
7n 151
IC 129
RtTg 243
MVeMC 167
Sp5C 122
7n 183
Sp5C 136
VCA 334
Pn 166
8n 115
7n 177
SNC 302
Amb 275
SNR 137
Pr5DM 312
7n

RMC 186
SNC 294
SNR 148
AP 203
SC 151
SNC 150
7n 167
3N 232
SC 149
IC 130
Sp5C 155
DC 129
SNR 164
Pr5VL 305
SC 154
IC 160
RtTg 224
7n 294
IC 151
SC 297
RtTg 211
VLL 138
Sol 157
RMC 275
DC 99
5N 283
IO 231
LRt 279
DC 131
7n 227
7n 277
IC 289
10N 208
Sp5I 312
IC 159
IC 243
VCA 97
7n 292
VLL 298
7n 175
RMC 203
IC 208
RMC 269
SC 186
8n 310
VLL 139
Pn 268
SC 152
DC 123
8n 122
8n 125
SC 252
SC 300
SNR 167
4N 228
10N 182
SNR 267
3N 230
IC 279
RtTg 181
3N 216
SNC 154
SNR 169
10N 204
VCA 96
LVe 138
RtTg 239
VCP 318
VLL 149
Tz 187
SNC 306
VCP 333
PBG 125
10N 178
DC 119
8n 116
Pn 178
Sp5I 310
DC 134
VCP 331
Sp5C 114
8n 320
SNC 145
Sp5C 134
7n 265
5N 285
Pn 194
7N 273
SNR 286
VCA 331
DC 285
Sp5I 288
SNR 323
LRt 160
Pr5VL 121
8n 105
7n 297
RtTg 179
VLL 130
SNR 275
5N 137
IC 249
IC 258
7n 233
Sp5I 309
SNC 167
7N 167
SNR 301
SNR 265
4N 205
IO 240
Pn 181
PBG 130
SpVe 160
7n 185
7n 243
LVe 140
Tz 230
12N 213
DC 334
12N 207
Tz 193
Pr5DM 315
SNR 324
IC 188
RMC 188
Tz 205
Sp5O 312
SC 191
VCA 95
5N 275
DC 

KeyboardInterrupt: 

In [179]:
merged_annotations_table = DataFrame(annotations.values())

In [180]:
merged_annotations_table

,class,creator,edits,filename,id,label_position,name,orientation,parent_structure,resolution,section,side,side_manually_assigned,time_created,type,vertices
61417973919587,contour,yuncong,"[{u'username': u'yuncong', u'timestamp': u'041...",CHATM3_slide83_2018_01_26-S2,61417973919587,"[45242.97334769556, 14458.899611503883]",10N,sagittal,[],raw,247,None,False,04162018154212,confirmed,"[[44355.51853031244, 14412.688257996695], [441..."
165498910605087,contour,yuncong,"[{u'username': u'yuncong', u'timestamp': u'041...",CHATM3_slide62_2018_02_19-S2,165498910605087,"[29560.777569723257, 16078.946725111617]",SNR,sagittal,[],raw,182,None,False,04162018151052,confirmed,"[[29458.24516597796, 16466.65042560262], [2915..."
110473719026558,contour,yuncong,"[{u'username': u'yuncong', u'timestamp': u'041...",CHATM3_slide66_2018_02_19-S1,110473719026558,"[34502.32822582653, 16157.73820339809]",RtTg,sagittal,[],raw,192,None,False,04162018151246,confirmed,"[[33282.32820358013, 15984.895809762857], [336..."
102790288330653,contour,yuncong,"[{u'username': u'yuncong', u'timestamp': u'041...",CHATM3_slide77_2018_01_26-S1,102790288330653,"[32229.89735960506, 6030.339041948793]",SC,sagittal,[],raw,227,None,False,04162018152920,confirmed,"[[31340.615265167744, 7394.858573806586], [309..."
108860727905611,contour,yuncong,"[{u'username': u'yuncong', u'timestamp': u'041...",CHATM3_slide77_2018_01_26-S1,108860727905611,"[35530.58904660281, 5787.660395539464]",IC,sagittal,[],raw,227,None,False,04162018152900,confirmed,"[[34730.80377764012, 5168.051308139539], [3468..."
131633584750329,contour,yuncong,"[{u'username': u'yuncong', u'timestamp': u'041...",CHATM3_slide70_2018_02_19-S1,131633584750329,"[34516.40779894652, 14570.824878276488]",6N,sagittal,[],raw,204,None,False,04162018151748,confirmed,"[[34248.00483041288, 14764.966211574136], [340..."
217866353503758,contour,yuncong,"[{u'username': u'yuncong', u'timestamp': u'041...",CHATM3_slide59_2018_02_19-S2,217866353503758,"[38807.11996040599, 10398.53216215258]",LC,sagittal,[],raw,175,None,False,04162018150357,confirmed,"[[38963.409612330135, 10057.157328010566], [38..."
146476015747094,contour,yuncong,"[{u'username': u'yuncong', u'timestamp': u'041...",CHATM3_slide69_2018_02_19-S2,146476015747094,"[33214.52316272273, 13546.70416079169]",6N,sagittal,[],raw,205,None,False,04162018151919,confirmed,"[[32923.39783832048, 13734.518908083017], [328..."
126673529461145,contour,yuncong,"[{u'username': u'yuncong', u'timestamp': u'041...",CHATM3_slide62_2018_02_19-S3,126673529461145,"[36777.61878491937, 18569.784908726448]",Tz,sagittal,[],raw,184,None,False,04162018150824,confirmed,"[[36280.710727115766, 18934.772876054507], [35..."
65942681262602,contour,yuncong,"[{u'username': u'yuncong', u'timestamp': u'041...",CHATM3_slide54_2018_02_18-S3,65942681262602,"[32927.618200539524, 7901.654884485934]",SC,sagittal,[],raw,160,None,False,04162018144513,confirmed,"[[32214.71748645316, 10149.322529429843], [316..."


In [ ]:
fp = DataManager.get_annotation_filepath(stack=stack, by_human=True, suffix='contours', timestamp='now')
save_hdf_v2(merged_annotations_table, fp)
upload_to_s3(fp)

In [34]:
contours_df, timestamp = DataManager.load_annotation_v4(stack=stack, by_human=True, timestamp='latest', 
                                                        suffix='contours', return_timestamp=True)

aws s3 cp --recursive "s3://mousebrainatlas-data/CSHL_labelings_v3/CHATM3" "/home/yuncong/CSHL_labelings_v3/CHATM3" --exclude "*" --include "*contours*"
0
latest timestamp:  04172018132221


In [35]:
for cid in contours_df.index:
    contours_df.at[cid, 'creator'] = 'lauren'
    
    edits = contours_df.loc[cid]['edits']
    new_edits = []
    for edit in edits:
        new_edit = edit.copy()
        new_edit['username'] = 'lauren'
        new_edits.append(new_edit)
        
    contours_df.at[cid, 'edits'] = new_edits

In [36]:
contours_df

,class,creator,edits,filename,id,label_position,name,orientation,parent_structure,resolution,section,side,side_manually_assigned,time_created,type,vertices
61417973919587,contour,lauren,"[{u'username': u'lauren', u'timestamp': u'0416...",CHATM3_slide83_2018_01_26-S2,61417973919587,"[45242.97334769556, 14458.899611503883]",10N,sagittal,[],raw,247,None,False,04162018154212,confirmed,"[[44355.51853031244, 14412.688257996695], [441..."
165498910605087,contour,lauren,"[{u'username': u'lauren', u'timestamp': u'0416...",CHATM3_slide62_2018_02_19-S2,165498910605087,"[29560.777569723257, 16078.946725111617]",SNR,sagittal,[],raw,182,None,False,04162018151052,confirmed,"[[29458.24516597796, 16466.65042560262], [2915..."
110473719026558,contour,lauren,"[{u'username': u'lauren', u'timestamp': u'0416...",CHATM3_slide66_2018_02_19-S1,110473719026558,"[34502.32822582653, 16157.73820339809]",RtTg,sagittal,[],raw,192,None,False,04162018151246,confirmed,"[[33282.32820358013, 15984.895809762857], [336..."
102790288330653,contour,lauren,"[{u'username': u'lauren', u'timestamp': u'0416...",CHATM3_slide77_2018_01_26-S1,102790288330653,"[32229.89735960506, 6030.339041948793]",SC,sagittal,[],raw,227,None,False,04162018152920,confirmed,"[[31340.615265167744, 7394.858573806586], [309..."
108860727905611,contour,lauren,"[{u'username': u'lauren', u'timestamp': u'0416...",CHATM3_slide77_2018_01_26-S1,108860727905611,"[35530.58904660281, 5787.660395539464]",IC,sagittal,[],raw,227,None,False,04162018152900,confirmed,"[[34730.80377764012, 5168.051308139539], [3468..."
131633584750329,contour,lauren,"[{u'username': u'lauren', u'timestamp': u'0416...",CHATM3_slide70_2018_02_19-S1,131633584750329,"[34516.40779894652, 14570.824878276488]",6N,sagittal,[],raw,204,None,False,04162018151748,confirmed,"[[34248.00483041288, 14764.966211574136], [340..."
217866353503758,contour,lauren,"[{u'username': u'lauren', u'timestamp': u'0416...",CHATM3_slide59_2018_02_19-S2,217866353503758,"[38807.11996040599, 10398.53216215258]",LC,sagittal,[],raw,175,None,False,04162018150357,confirmed,"[[38963.409612330135, 10057.157328010566], [38..."
146476015747094,contour,lauren,"[{u'username': u'lauren', u'timestamp': u'0416...",CHATM3_slide69_2018_02_19-S2,146476015747094,"[33214.52316272273, 13546.70416079169]",6N,sagittal,[],raw,205,None,False,04162018151919,confirmed,"[[32923.39783832048, 13734.518908083017], [328..."
126673529461145,contour,lauren,"[{u'username': u'lauren', u'timestamp': u'0416...",CHATM3_slide62_2018_02_19-S3,126673529461145,"[36777.61878491937, 18569.784908726448]",Tz,sagittal,[],raw,184,None,False,04162018150824,confirmed,"[[36280.710727115766, 18934.772876054507], [35..."
65942681262602,contour,lauren,"[{u'username': u'lauren', u'timestamp': u'0416...",CHATM3_slide54_2018_02_18-S3,65942681262602,"[32927.618200539524, 7901.654884485934]",SC,sagittal,[],raw,160,None,False,04162018144513,confirmed,"[[32214.71748645316, 10149.322529429843], [316..."


In [37]:
fp = DataManager.get_annotation_filepath(stack=stack, by_human=True, suffix='contours', timestamp='now')
save_hdf_v2(contours_df, fp)
upload_to_s3(fp)

aws s3 cp "/home/yuncong/CSHL_labelings_v3/CHATM3/CHATM3_annotation_contours_04252018112445.hdf" "s3://mousebrainatlas-data/CSHL_labelings_v3/CHATM3/CHATM3_annotation_contours_04252018112445.hdf"
0
